<a href="https://colab.research.google.com/github/Dipeshpal/Intent-Classification-small-transformers/blob/main/Intent_Classification_small_Jarvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A: Prepare a 🤗 Hugging Face Dataset from Data from CSV Format

In [4]:
!pip install datasets transformers shutup ipython-autotime evaluate shutup --quiet

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
dataset_file = 'dataset-filtered.csv'
model_dipesh = 'Intent-Classification-small'
tokenizer_dipesh = 'Intent-Classification-small'
dataset_dipesh = 'Intent-Classification-small'
per_device_train_batch_size=64
per_device_eval_batch_size=64
num_train_epochs=8
base_model = 'joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english'

In [8]:
%load_ext autotime
import shutup
shutup.please()

time: 5.45 ms (started: 2023-01-27 22:07:56 +00:00)


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

pd.set_option('max_colwidth', 400)

time: 1.92 s (started: 2023-01-27 22:07:57 +00:00)


In [10]:
base_path = '/content/drive/MyDrive/Machine_Learning/Transformers-Classification/'

time: 8.09 ms (started: 2023-01-27 22:07:59 +00:00)


In [11]:
dataset = load_dataset("csv", data_files=base_path+dataset_file)

  0%|          | 0/1 [00:00<?, ?it/s]

time: 1.05 s (started: 2023-01-27 22:07:59 +00:00)


In [12]:
dataset = dataset["train"].shuffle(seed=42)

time: 13.2 ms (started: 2023-01-27 22:08:00 +00:00)


In [13]:
# dataset = dataset.rename_column(
#     original_column_name="labels", new_column_name="label"
# )
dataset = dataset.remove_columns('labels')

time: 9.21 ms (started: 2023-01-27 22:08:00 +00:00)


In [14]:
dataset = dataset.train_test_split(train_size=0.9, seed=42)
# Rename the default "test" split to "validation"
dataset["validation"] = dataset.pop("test")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 684
    })
})

time: 20.5 ms (started: 2023-01-27 22:08:00 +00:00)


In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)

dataset = dataset.map(lambda examples: tokenizer(examples["text"], return_tensors="np"), batched=True)

time: 5.34 s (started: 2023-01-27 22:08:00 +00:00)


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
})

time: 4.65 ms (started: 2023-01-27 22:08:06 +00:00)


### Generating id2label, label2id and num_labels

In [17]:
from datasets import Dataset, Value, ClassLabel, Features

# Creating a ClassLabel Object
df = dataset["train"].to_pandas()
labels = df['intent'].unique().tolist()
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['intent'])
    return example

dataset = dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 315 ms (started: 2023-01-27 22:08:06 +00:00)


In [18]:
# dataset = dataset.align_labels_with_mapping(label2id, "label")

time: 357 µs (started: 2023-01-27 22:08:06 +00:00)


In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 4.35 ms (started: 2023-01-27 22:08:06 +00:00)


In [20]:
dataset['train'][0]

{'text': 'Thank you, goodbye',
 'intent': 'goodbye',
 'input_ids': [101, 4067, 2017, 1010, 9119, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1],
 'label': 0}

time: 4.47 ms (started: 2023-01-27 22:08:06 +00:00)


In [21]:
dataset.push_to_hub(f"dipesh/{dataset_dipesh}")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

time: 20.4 s (started: 2023-01-27 22:08:06 +00:00)


# PART B: Hugging Face Tokenization: Use a Pre-trained Tokenizer or Train a New Tokenizer from scratch?

As suggested in the official documentation, if a language model is not available in the language you are interested in, or if your corpus is very different from the one your language model was trained on, you will most likely want to retrain the model from scratch using a tokenizer adapted to your data. That will require training a new tokenizer on your dataset.

## Using Pretrained

In [22]:
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)

outputs = pretrained_tokenizer(
    dataset["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print("vocab_size: ", len(pretrained_tokenizer))

Input IDs length: 2
Input chunk lengths: [6, 9]
vocab_size:  30522
time: 950 ms (started: 2023-01-27 22:08:33 +00:00)


In [23]:
txt = "how are you"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)

[101, 2129, 2024, 2017, 102]
time: 749 µs (started: 2023-01-27 22:08:34 +00:00)


In [24]:
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 945 µs (started: 2023-01-27 22:08:34 +00:00)


## Train a new tokenizer

In [25]:
def get_training_corpus():
    batch_size = 1000
    return (
        dataset["train"][i : i + batch_size]["text"]
        for i in range(0, len(dataset["train"]), batch_size)
    )
training_corpus = get_training_corpus()

time: 776 µs (started: 2023-01-27 22:08:35 +00:00)


In [26]:
for data in get_training_corpus():
    print(len(data))

1000
1000
1000
1000
1000
1000
153
time: 114 ms (started: 2023-01-27 22:08:36 +00:00)


### NOTE: Now that we have our corpus in the form of an iterator of batches of texts, we are ready to train a new tokenizer. To do this, we first need to load the tokenizer we want to pair with our model

### NOTE: Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as facebook/bart-large-mnli, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.

In [27]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)
vocab_size = len(pretrained_tokenizer)
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

time: 1.46 s (started: 2023-01-27 22:08:37 +00:00)


In [28]:
print(tokenizer.eos_token_id, ",", tokenizer.vocab_size)

None , 3424
time: 2.4 ms (started: 2023-01-27 22:08:39 +00:00)


In [29]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print(tokens)

[2, 180, 145, 88, 3]
time: 1.1 ms (started: 2023-01-27 22:08:39 +00:00)


In [30]:
converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 805 µs (started: 2023-01-27 22:08:39 +00:00)


In [31]:
print("pretrained_tokenizer-")
print(pretrained_tokenizer)

print("New tokenizer-")
print(tokenizer)

pretrained_tokenizer-
DistilBertTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
New tokenizer-
DistilBertTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=3424, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
time: 866 µs (started: 2023-01-27 22:08:39 +00:00)


In [32]:
print("New tokenizer: ", len(tokenizer.tokenize(txt)))
print("Pretrained tokenizer: ", len(pretrained_tokenizer.tokenize(txt)))

New tokenizer:  3
Pretrained tokenizer:  3
time: 1.24 ms (started: 2023-01-27 22:08:39 +00:00)


### Saving tokenizer in local

In [33]:
path=f"/"
file_name=model_dipesh
tokenizer.save_pretrained(path+file_name)

('/Intent-Classification-small/tokenizer_config.json',
 '/Intent-Classification-small/special_tokens_map.json',
 '/Intent-Classification-small/vocab.txt',
 '/Intent-Classification-small/added_tokens.json',
 '/Intent-Classification-small/tokenizer.json')

time: 10.6 ms (started: 2023-01-27 22:08:42 +00:00)


### Loading tokenizer from local

In [34]:
loaded_tokenizer = AutoTokenizer.from_pretrained(path+file_name)

time: 4.06 ms (started: 2023-01-27 22:08:45 +00:00)


In [35]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)

trained tokenizer: [2, 180, 145, 88, 3]
loaded tokenizer: [2, 180, 145, 88, 3]
time: 4.78 ms (started: 2023-01-27 22:08:45 +00:00)


In [36]:
tokenizer.push_to_hub(f"dipesh/{tokenizer_dipesh}")

CommitInfo(commit_url='https://huggingface.co/dipesh/Intent-Classification-small/commit/b796353e8938d6e104cf11ed86366990e2d979f1', commit_message='Upload tokenizer', commit_description='', oid='b796353e8938d6e104cf11ed86366990e2d979f1', pr_url=None, pr_revision=None, pr_num=None)

time: 5.16 s (started: 2023-01-27 22:08:48 +00:00)


You can then load the tokenizer from anywhere with the from_pretrained() method:

In [37]:
downloaded_tokenizer = AutoTokenizer.from_pretrained(f"dipesh/{model_dipesh}")

txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained (New) tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)
tokens = downloaded_tokenizer(txt)['input_ids']
print("downloaded tokenizer:", tokens)

trained (New) tokenizer: [2, 180, 145, 88, 3]
loaded tokenizer: [2, 180, 145, 88, 3]
downloaded tokenizer: [2, 180, 145, 88, 3]
time: 7.84 s (started: 2023-01-27 22:08:56 +00:00)


# PART C: Training a model

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizerFast, RobertaTokenizer
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification, \
    TFRobertaForSequenceClassification, TFAutoModel, AutoModel, AutoModelForSequenceClassification
from sklearn import preprocessing
from datasets import load_dataset

time: 858 ms (started: 2023-01-27 22:09:08 +00:00)


### Loading Dataset

In [39]:
f'dipesh/{dataset_dipesh}'

'dipesh/Intent-Classification-small'

time: 3.17 ms (started: 2023-01-27 22:09:10 +00:00)


In [40]:
dataset = load_dataset(f'dipesh/Intent-Classification-small')
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6153 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/684 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/dipesh___parquet/dipesh--Intent-Classification-small-da45acdc519702ba/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 14.6 s (started: 2023-01-27 22:09:13 +00:00)


### Creating Datacollector and evaluator

In [41]:
from transformers import DataCollatorWithPadding
import evaluate


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

time: 2.6 s (started: 2023-01-27 22:09:33 +00:00)


In [42]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

time: 1.16 ms (started: 2023-01-27 22:09:42 +00:00)


### Loading Model

In [43]:
def id2label_gen(id):
    return ClassLabels.int2str(id)

def label2id_gen(label):
    return ClassLabels.str2int(label)

total_labels = ClassLabels.num_classes

print(id2label_gen(3), label2id_gen(id2label_gen(3)),  "total_labels: ", total_labels)

covid cases 3 total_labels:  21
time: 7.48 ms (started: 2023-01-27 22:09:43 +00:00)


In [44]:
id2label = {i: ClassLabels.int2str(i) for i in range(total_labels)}

label2id = {ClassLabels.int2str(i): i for i in range(total_labels)}
id2label, label2id

({0: 'goodbye',
  1: 'volume control',
  2: 'play games',
  3: 'covid cases',
  4: 'open website',
  5: 'tell me joke',
  6: 'play on youtube',
  7: 'places near me',
  8: 'greet and hello hi kind of things, general check in',
  9: 'asking time',
  10: 'asking date',
  11: 'tell me news',
  12: 'asking weather',
  13: 'download youtube video',
  14: 'what can you do',
  15: 'take screenshot',
  16: 'send email',
  17: 'i am bored',
  18: 'click photo',
  19: 'tell me about',
  20: 'send whatsapp message'},
 {'goodbye': 0,
  'volume control': 1,
  'play games': 2,
  'covid cases': 3,
  'open website': 4,
  'tell me joke': 5,
  'play on youtube': 6,
  'places near me': 7,
  'greet and hello hi kind of things, general check in': 8,
  'asking time': 9,
  'asking date': 10,
  'tell me news': 11,
  'asking weather': 12,
  'download youtube video': 13,
  'what can you do': 14,
  'take screenshot': 15,
  'send email': 16,
  'i am bored': 17,
  'click photo': 18,
  'tell me about': 19,
  'send 

time: 8.04 ms (started: 2023-01-27 22:09:43 +00:00)


In [45]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# If not loaded then load tokenizer by uncommenting below line
# tokenizer = AutoTokenizer.from_pretrained("dipesh/Intent-Classification-Commands-balanced-ds-large")
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=total_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
# model = AutoModelForSequenceClassification.from_pretrained(base_model, ignore_mismatched_sizes=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([60, 768]) in the checkpoint and torch.Size([21, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([60]) in the checkpoint and torch.Size([21]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 2.11 s (started: 2023-01-27 22:09:45 +00:00)


### Creating tokenized dataset

In [46]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

time: 295 ms (started: 2023-01-27 22:09:47 +00:00)


In [47]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 3.28 ms (started: 2023-01-27 22:09:49 +00:00)


### Finetuning Model

In [48]:
config = {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": id2label,
  "initializer_range": 0.02,
  "label2id": label2id,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": False,
  "tie_weights_": True,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

time: 1.4 ms (started: 2023-01-27 22:09:51 +00:00)


In [49]:
import json

with open(path+file_name+'/config.json', "w") as f:
    json.dump(config, f)

time: 1.37 ms (started: 2023-01-27 22:09:52 +00:00)


In [50]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(base_model)
config.update({"id2label": id2label})
config.update({"label2id": label2id})
# Push the config to your namespace with the name "my-finetuned-bert".
config.push_to_hub(f"dipesh/{model_dipesh}")

CommitInfo(commit_url='https://huggingface.co/dipesh/Intent-Classification-small/commit/d47ff4d5fca3aa29b3a97c0a6ba2dd10882b27b8', commit_message='Upload config', commit_description='', oid='d47ff4d5fca3aa29b3a97c0a6ba2dd10882b27b8', pr_url=None, pr_revision=None, pr_num=None)

time: 4.32 s (started: 2023-01-27 22:09:54 +00:00)


In [51]:
# import os
# os.system('git init /content/Intent-Classification-small')

time: 378 µs (started: 2023-01-27 22:09:59 +00:00)


In [52]:
output_dir_model_hf_name=f"dipesh/{model_dipesh}"

training_args = TrainingArguments(
    output_dir=output_dir_model_hf_name,
    learning_rate=2e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/dipesh/Intent-Classification-small into local empty directory.
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, intent. If text, intent are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6153
  Num Epochs = 8
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 776
  Number of trainable parameters = 66969621
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.110223,0.387427
2,No log,1.426596,0.654971
3,No log,0.865460,0.811404
4,No log,0.627161,0.862573
5,No log,0.475459,0.893275
6,1.391900,0.358653,0.915205
7,1.391900,0.332141,0.916667
8,1.391900,0.316937,0.922515


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, intent. If text, intent are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 684
  Batch size = 64
Saving model checkpoint to dipesh/Intent-Classification-small/checkpoint-97
Configuration saved in dipesh/Intent-Classification-small/checkpoint-97/config.json
Model weights saved in dipesh/Intent-Classification-small/checkpoint-97/pytorch_model.bin
tokenizer config file saved in dipesh/Intent-Classification-small/checkpoint-97/tokenizer_config.json
Special tokens file saved in dipesh/Intent-Classification-small/checkpoint-97/special_tokens_map.json
tokenizer config file saved in dipesh/Intent-Classification-small/tokenizer_config.json
Special tokens file saved in dipesh/Intent-Classification-small/special_tokens_map.json
The following 

TrainOutput(global_step=776, training_loss=1.0311882176350073, metrics={'train_runtime': 161.9872, 'train_samples_per_second': 303.876, 'train_steps_per_second': 4.791, 'total_flos': 203182482107970.0, 'train_loss': 1.0311882176350073, 'epoch': 8.0})

time: 2min 54s (started: 2023-01-27 22:09:59 +00:00)


# Inference

In [53]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=f"./{output_dir_model_hf_name}")

loading configuration file ./dipesh/Intent-Classification-small/config.json
Model config DistilBertConfig {
  "_name_or_path": "./dipesh/Intent-Classification-small",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "goodbye",
    "1": "volume control",
    "2": "play games",
    "3": "covid cases",
    "4": "open website",
    "5": "tell me joke",
    "6": "play on youtube",
    "7": "places near me",
    "8": "greet and hello hi kind of things, general check in",
    "9": "asking time",
    "10": "asking date",
    "11": "tell me news",
    "12": "asking weather",
    "13": "download youtube video",
    "14": "what can you do",
    "15": "take screenshot",
    "16": "send email",
    "17": "i am bored",
    "18": "click photo",
    "19": "tell me about",
    "20": "send whatsapp message"
  },
  "initializer_range": 0.02,
  "label2id": {

time: 1.24 s (started: 2023-01-27 22:12:53 +00:00)


In [54]:
classifier("click photo")

[{'label': 'click photo', 'score': 0.80433589220047}]

time: 230 ms (started: 2023-01-27 22:12:54 +00:00)


In [55]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(
    f"dipesh/{tokenizer_dipesh}")
model = AutoModelForSequenceClassification.from_pretrained(
    f"dipesh/{model_dipesh}")
classifier_return_all_scores = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True )
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file dipesh/Intent-Classification-small/config.json
Model config DistilBertConfig {
  "_name_or_path": "dipesh/Intent-Classification-small",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "goodbye",
    "1": "volume control",
    "2": "play games",
    "3": "covid cases",
    "4": "open website",
    "5": "tell me joke",
    "6": "play on youtube",
    "7": "places near me",
    "8": "greet and hello hi kind of things, general check in",
    "9": "asking time",
    "10": "asking date",
    "11": "tell me news",
    "12": "asking weather",
    "13": "download youtube video",
    "14": "what can you do",
    "15": "take screenshot",
    "16": "send email",
    "17":

time: 807 ms (started: 2023-01-27 22:12:54 +00:00)


In [56]:
sequence = "make me laugh"

# Classify the sequence
results = classifier(sequence)

# Print the results
print(results)

[{'label': 'tell me joke', 'score': 0.9713832139968872}]
time: 46.2 ms (started: 2023-01-27 22:12:55 +00:00)


In [57]:
sequence = "make me laugh"

# Classify the sequence
results = classifier_return_all_scores(sequence)

# Print the results
print(results)

[[{'label': 'goodbye', 'score': 0.001874322071671486}, {'label': 'volume control', 'score': 0.0008814579923637211}, {'label': 'play games', 'score': 0.0016746718902140856}, {'label': 'covid cases', 'score': 0.0005680847098119557}, {'label': 'open website', 'score': 0.0008297894382849336}, {'label': 'tell me joke', 'score': 0.9713832139968872}, {'label': 'play on youtube', 'score': 0.0031239702366292477}, {'label': 'places near me', 'score': 0.0016206579748541117}, {'label': 'greet and hello hi kind of things, general check in', 'score': 0.002956580137833953}, {'label': 'asking time', 'score': 0.001606754376552999}, {'label': 'asking date', 'score': 0.0006566939991898835}, {'label': 'tell me news', 'score': 0.00135100691113621}, {'label': 'asking weather', 'score': 0.0008501193369738758}, {'label': 'download youtube video', 'score': 0.0005298893083818257}, {'label': 'what can you do', 'score': 0.0010970052098855376}, {'label': 'take screenshot', 'score': 0.0013030756963416934}, {'label'

time: 55.7 ms (started: 2023-01-27 22:12:55 +00:00)
